In [1]:
import numpy as np
import random
import networkx as nx
import pandas as pd
from sklearn.metrics import jaccard_score
from pandas.core.frame import DataFrame
import heapq
import time

In [2]:
data = pd.read_csv("data.csv")
MSH = pd.read_csv("MSH.csv")
selected_node = pd.read_csv("selected_node.csv")

In [3]:
Characteristic_Graph = nx.Graph()
for i in range(len(data)):
    Characteristic_Graph.add_edge(data['0'][i]-1,data['1'][i]-1)
Connection_Matrix=np.array(nx.adjacency_matrix(Characteristic_Graph).todense())
Verte = list(Characteristic_Graph.nodes)

C:\Users\HUAWEI\AppData\Local\Temp\ipykernel_15096\2368657692.py:4: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  Connection_Matrix=np.array(nx.adjacency_matrix(Characteristic_Graph).todense())


In [4]:
def difference_less_than_one(a,b):
    return np.abs(a-b).sum()

In [5]:
def jaccard(A,B):
    bigcap = len(list(set(A) & set(B)))
    bigcup = len(list(set(A) | set(B)))
    jaccard = bigcap/bigcup
    return jaccard

In [6]:
def similarest_ten(node,Verte):
    global Characteristic_Graph
    max_index = []
    Real_Jaccard = pd.DataFrame(columns=['node','jaccard'],dtype=float)
    for i in Verte:
        df_1 = pd.DataFrame([[i,jaccard(list(nx.all_neighbors(Characteristic_Graph, node)),list(nx.all_neighbors(Characteristic_Graph, i)))]],columns=['node','jaccard'],dtype=float)
        Real_Jaccard = pd.concat([Real_Jaccard,df_1], ignore_index=True)
    Real_Jaccard=Real_Jaccard.sort_values(by="jaccard" , ascending=False)
    print("Jaccard")
    if len(Real_Jaccard)>0:
        print(Real_Jaccard)
    Real_Jaccard_list = np.array(Real_Jaccard).tolist()
    max_ten = 10
    if max_ten>len(Real_Jaccard_list):
        max_ten=len(Real_Jaccard_list)
    for i in range(max_ten):
        max_index.append(Real_Jaccard_list[i][0])
    if (len(max_index)<10):
        while (len(max_index)<10):
            max_index.append(-1)
    return max_index

In [7]:
def LSH(node,b,r,bound):# b个行条，每行r个哈希函数
    global MSH
    global G
    global Verte
    MSH_tobe_Verteilung = np.array(MSH.iloc[:, 0:b*r])
    volunteer = []
    MSH_Verteilung = np.array_split(MSH_tobe_Verteilung, b, axis=1)
    print("MSH_Verteilung")
    print(MSH_Verteilung[0][node])
    for i in range (len(MSH_Verteilung)):
        for j in range (len(Verte)):
            if (difference_less_than_one(MSH_Verteilung[i][j],MSH_Verteilung[i][node])<=bound) & (j!=node) & (j not in volunteer):
                volunteer.append(j)
    print("volunteer")
    print(volunteer)
    return similarest_ten(node,volunteer)

In [8]:
test_node = []
for i in range (5):
    test_node+=[int(selected_node['node'][i])]

In [9]:
def conduct(b,r,bound):
    global test_node
    global Verte
    df = pd.DataFrame(columns=['node','accuracy','time','blank_control'],dtype=float)
    for i in range (5):
        print("-------------------")
        print("i")
        print(i)
        t_a=time.perf_counter()
        A = LSH(test_node[i],b,r,bound)
        t_b=time.perf_counter()
        Verte_without_i = list(Characteristic_Graph.nodes)
        if test_node[i] in Verte_without_i:
            Verte_without_i.remove(test_node[i])
        t_c=time.perf_counter()
        B = similarest_ten(test_node[i],Verte_without_i)
        t_d=time.perf_counter()
        accuracy = len(list(set(A) & set(B)))/len(B)
        run_time = t_b-t_a
        compare_time = t_d-t_c
        df1 = pd.DataFrame([[test_node[i],accuracy,run_time,compare_time]],columns=['node','accuracy','time','blank_control'],dtype=float)
        df = pd.concat([df,df1], ignore_index=True)
        print("A")
        print(A)
        print("B")
        print(B)
    print(df)

In [10]:
conduct(25,4,1)

-------------------
i
0
MSH_Verteilung
[12866.   158.  5548.  8645.]
volunteer
[1616, 1629, 1342, 1623, 1593, 1611, 781, 1606, 1612, 1600, 2470, 1352, 1216, 1267, 1286, 1290, 1296, 1304, 1320, 1322, 1324, 1326, 1329, 1330, 1335, 1340, 1346, 1351, 1355, 1375, 1369, 1282, 1285, 1350, 2468, 10485, 10486, 10487, 10488, 10489, 10490]
Jaccard
       node   jaccard
0    1616.0  0.837209
4    1593.0  0.787234
1    1629.0  0.731707
8    1612.0  0.705882
3    1623.0  0.701754
9    1600.0  0.655738
21   1326.0  0.645161
5    1611.0  0.642857
11   1352.0  0.642857
16   1296.0  0.606061
27   1351.0  0.606061
17   1304.0  0.606061
15   1290.0  0.606061
25   1340.0  0.597015
18   1320.0  0.597015
29   1375.0  0.584906
14   1286.0  0.579710
34   2468.0  0.574468
13   1267.0  0.566038
28   1355.0  0.563636
26   1346.0  0.555556
2    1342.0  0.547945
12   1216.0  0.519481
22   1329.0  0.516667
35  10485.0  0.511111
31   1282.0  0.490566
36  10486.0  0.488889
37  10487.0  0.488889
39  10489.0  0.488889
4

In [11]:
conduct(50,2,1)

-------------------
i
0
MSH_Verteilung
[12866.   158.]
volunteer
[1616, 1629, 1267, 1352, 1355, 1361, 1375, 1329, 1342, 1593, 1612, 1200, 1216, 1276, 1279, 1286, 1290, 1296, 1304, 1319, 1320, 1322, 1326, 1338, 1340, 1346, 1351, 1623, 1606, 10486, 10487, 10489, 10490, 1611, 2468, 791, 803, 800, 1600, 1303, 1343, 240, 1285, 1324, 1335, 767, 804, 1273, 1282, 765, 2470, 781, 1278, 222, 239, 763, 776, 777, 801, 1330, 1350, 1610, 10485, 10488, 1266, 214, 772, 1179, 1369, 219, 221, 530, 866, 1146, 1297, 1314, 1844, 1883, 6326, 10372, 10375, 10376, 10377, 10378, 3556, 11024, 11025, 11669, 11905, 11910, 11911, 11912, 11913, 11914, 11915, 11917, 11918, 11919, 11920, 11921, 1832, 1852, 1860, 1869, 1871, 1873, 1881, 1895, 1896]
Jaccard
       node   jaccard
0    1616.0  0.837209
9    1593.0  0.787234
1    1629.0  0.731707
10   1612.0  0.705882
27   1623.0  0.701754
..      ...       ...
90  11911.0  0.014706
93  11914.0  0.014706
88  11905.0  0.013889
86  11025.0  0.012500
85  11024.0  0.012500

[

In [12]:
conduct(25,2,1)

-------------------
i
0
MSH_Verteilung
[12866.   158.]
volunteer
[1616, 1629, 1267, 1352, 1355, 1361, 1375, 1329, 1342, 1593, 1612, 1200, 1216, 1276, 1279, 1286, 1290, 1296, 1304, 1319, 1320, 1322, 1326, 1338, 1340, 1346, 1351, 1623, 1606, 10486, 10487, 10489, 10490, 1611, 2468, 791, 803, 800, 1600, 1303, 1343, 240, 1285, 1324, 1335, 767, 804, 1273, 1282, 765, 2470, 781]
Jaccard
       node   jaccard
0    1616.0  0.837209
9    1593.0  0.787234
1    1629.0  0.731707
10   1612.0  0.705882
27   1623.0  0.701754
38   1600.0  0.655738
22   1326.0  0.645161
3    1352.0  0.642857
33   1611.0  0.642857
16   1290.0  0.606061
18   1304.0  0.606061
17   1296.0  0.606061
26   1351.0  0.606061
20   1320.0  0.597015
24   1340.0  0.597015
6    1375.0  0.584906
23   1338.0  0.579710
15   1286.0  0.579710
14   1279.0  0.579710
34   2468.0  0.574468
19   1319.0  0.571429
2    1267.0  0.566038
4    1355.0  0.563636
25   1346.0  0.555556
8    1342.0  0.547945
45    767.0  0.533333
36    803.0  0.524590
35

In [13]:
conduct(75,2,1)

-------------------
i
0
MSH_Verteilung
[12866.   158.]
volunteer
[1616, 1629, 1267, 1352, 1355, 1361, 1375, 1329, 1342, 1593, 1612, 1200, 1216, 1276, 1279, 1286, 1290, 1296, 1304, 1319, 1320, 1322, 1326, 1338, 1340, 1346, 1351, 1623, 1606, 10486, 10487, 10489, 10490, 1611, 2468, 791, 803, 800, 1600, 1303, 1343, 240, 1285, 1324, 1335, 767, 804, 1273, 1282, 765, 2470, 781, 1278, 222, 239, 763, 776, 777, 801, 1330, 1350, 1610, 10485, 10488, 1266, 214, 772, 1179, 1369, 219, 221, 530, 866, 1146, 1297, 1314, 1844, 1883, 6326, 10372, 10375, 10376, 10377, 10378, 3556, 11024, 11025, 11669, 11905, 11910, 11911, 11912, 11913, 11914, 11915, 11917, 11918, 11919, 11920, 11921, 1832, 1852, 1860, 1869, 1871, 1873, 1881, 1895, 1896, 1074, 1168, 216, 757, 1190, 1193, 1196, 1227, 1231, 1812, 3639, 8650, 8658, 232]
Jaccard
       node   jaccard
0    1616.0  0.837209
9    1593.0  0.787234
1    1629.0  0.731707
10   1612.0  0.705882
27   1623.0  0.701754
..      ...       ...
98  11920.0  0.014706
90  11911

In [14]:
conduct(75,3,1)

-------------------
i
0
MSH_Verteilung
[12866.   158.  5548.]
volunteer
[1616, 1629, 1267, 1352, 1355, 1375, 1593, 1623, 1279, 1290, 1296, 1304, 1319, 1320, 1326, 1329, 1338, 1340, 1346, 1351, 1611, 1612, 791, 803, 2468, 240, 1285, 1322, 1324, 1335, 1342, 1273, 781, 1606, 222, 239, 767, 777, 1200, 1278, 1286, 1600, 2470, 804, 1179, 1369, 1330, 1610, 1216, 1282, 1350, 10485, 10486, 10487, 10488, 10489, 10490, 765, 801, 1343, 772, 776, 1303, 1276, 214]
Jaccard
      node   jaccard
0   1616.0  0.837209
6   1593.0  0.787234
1   1629.0  0.731707
21  1612.0  0.705882
7   1623.0  0.701754
..     ...       ...
44  1179.0  0.357143
64   214.0  0.350877
39  1278.0  0.341880
34   222.0  0.310078
31  1273.0  0.142857

[65 rows x 2 columns]
Jaccard
          node   jaccard
4518    1616.0  0.837209
4515    1593.0  0.787234
4520    1629.0  0.731707
4517    1612.0  0.705882
4519    1623.0  0.701754
...        ...       ...
6536    3003.0  0.000000
6537    3123.0  0.000000
6538    3134.0  0.000000
6539

In [15]:
conduct(25,3,1)

-------------------
i
0
MSH_Verteilung
[12866.   158.  5548.]
volunteer
[1616, 1629, 1267, 1352, 1355, 1375, 1593, 1623, 1279, 1290, 1296, 1304, 1319, 1320, 1326, 1329, 1338, 1340, 1346, 1351, 1611, 1612, 791, 803, 2468, 240, 1285, 1322, 1324, 1335, 1342, 1273, 781, 1606, 222, 239, 767, 777, 1200, 1278, 1286, 1600, 2470, 804, 1179, 1369, 1330, 1610]
Jaccard
      node   jaccard
0   1616.0  0.837209
6   1593.0  0.787234
1   1629.0  0.731707
21  1612.0  0.705882
7   1623.0  0.701754
41  1600.0  0.655738
14  1326.0  0.645161
20  1611.0  0.642857
3   1352.0  0.642857
19  1351.0  0.606061
10  1296.0  0.606061
11  1304.0  0.606061
9   1290.0  0.606061
13  1320.0  0.597015
17  1340.0  0.597015
5   1375.0  0.584906
8   1279.0  0.579710
40  1286.0  0.579710
16  1338.0  0.579710
24  2468.0  0.574468
12  1319.0  0.571429
2   1267.0  0.566038
4   1355.0  0.563636
18  1346.0  0.555556
30  1342.0  0.547945
47  1610.0  0.547945
36   767.0  0.533333
23   803.0  0.524590
22   791.0  0.524590
15  1329.0

In [16]:
conduct(50,3,1)

-------------------
i
0
MSH_Verteilung
[12866.   158.  5548.]
volunteer
[1616, 1629, 1267, 1352, 1355, 1375, 1593, 1623, 1279, 1290, 1296, 1304, 1319, 1320, 1326, 1329, 1338, 1340, 1346, 1351, 1611, 1612, 791, 803, 2468, 240, 1285, 1322, 1324, 1335, 1342, 1273, 781, 1606, 222, 239, 767, 777, 1200, 1278, 1286, 1600, 2470, 804, 1179, 1369, 1330, 1610, 1216, 1282, 1350, 10485, 10486, 10487, 10488, 10489, 10490, 765, 801, 1343, 772, 776, 1303]
Jaccard
      node   jaccard
0   1616.0  0.837209
6   1593.0  0.787234
1   1629.0  0.731707
21  1612.0  0.705882
7   1623.0  0.701754
..     ...       ...
59  1343.0  0.391304
44  1179.0  0.357143
39  1278.0  0.341880
34   222.0  0.310078
31  1273.0  0.142857

[63 rows x 2 columns]
Jaccard
          node   jaccard
4518    1616.0  0.837209
4515    1593.0  0.787234
4520    1629.0  0.731707
4517    1612.0  0.705882
4519    1623.0  0.701754
...        ...       ...
6536    3003.0  0.000000
6537    3123.0  0.000000
6538    3134.0  0.000000
6539    9081.0 

In [17]:
conduct(25,1,1)

-------------------
i
0
MSH_Verteilung
[12866.]
volunteer
[1267, 1325, 1329, 1352, 1355, 1361, 1375, 1593, 1616, 1629, 2406, 2468, 12610, 12611, 214, 216, 222, 240, 767, 1200, 1216, 1266, 1276, 1278, 1279, 1285, 1286, 1290, 1296, 1303, 1304, 1319, 1320, 1322, 1326, 1330, 1338, 1340, 1342, 1346, 1351, 1369, 1606, 1610, 1612, 1623, 1808, 4280, 6477, 8007, 11731, 11732, 11733, 11734, 11742, 1282, 1611, 10375, 10376, 2022, 5576, 7593, 10374, 10378, 12806, 239, 587, 598, 730, 765, 772, 776, 777, 781, 791, 800, 801, 803, 804, 840, 1162, 1203, 1208, 1233, 1318, 1324, 1335, 1600, 1664, 3763, 3765, 8650, 8652, 8653, 10486, 10487, 10489, 10490, 474, 5423, 10371, 10372, 10373, 10377, 17109, 16952, 17196, 1170, 14303, 1607, 1613, 1618, 1626, 802, 866, 1155, 2396, 2405, 2408, 5885, 219, 763, 820, 1343, 1350, 1794, 1832, 1852, 1860, 1864, 1869, 1871, 1873, 1881, 1895, 1896, 1992, 1993, 2470, 3751, 4717, 5598, 5599, 5604, 5613, 10485, 10488, 10491, 10492, 1297, 3817, 4773, 6326, 6392, 8463, 9065, 181

Jaccard
         node   jaccard
114    5842.0  0.343137
48     4121.0  0.320755
113    5838.0  0.317757
147    2592.0  0.305195
100     586.0  0.286885
...       ...       ...
678   10927.0  0.000000
632    1785.0  0.000000
631    1784.0  0.000000
586      99.0  0.000000
1253  18367.0  0.000000

[1254 rows x 2 columns]
Jaccard
          node   jaccard
3094    5842.0  0.343137
3658    4121.0  0.320755
3092    5838.0  0.317757
2723    2592.0  0.305195
2248     586.0  0.286885
...        ...       ...
7839    7778.0  0.000000
7840    1388.0  0.000000
7841    1390.0  0.000000
7842     821.0  0.000000
18769  18770.0  0.000000

[18770 rows x 2 columns]
A
[5842.0, 4121.0, 5838.0, 2592.0, 586.0, 3367.0, 3875.0, 3358.0, 2599.0, 1039.0]
B
[5842.0, 4121.0, 5838.0, 2592.0, 586.0, 3367.0, 3875.0, 3358.0, 2599.0, 1039.0]
-------------------
i
2
MSH_Verteilung
[14572.]
volunteer
[3825, 3829, 3834, 3896, 8133, 3868, 3886, 3887, 6841, 7750, 8128, 8129, 8130, 8132, 8134, 8135, 8136, 8137, 8384, 11909, 3

In [18]:
conduct(50,4,1)

-------------------
i
0
MSH_Verteilung
[12866.   158.  5548.  8645.]
volunteer
[1616, 1629, 1342, 1623, 1593, 1611, 781, 1606, 1612, 1600, 2470, 1352, 1216, 1267, 1286, 1290, 1296, 1304, 1320, 1322, 1324, 1326, 1329, 1330, 1335, 1340, 1346, 1351, 1355, 1375, 1369, 1282, 1285, 1350, 2468, 10485, 10486, 10487, 10488, 10489, 10490, 240, 765, 791, 803, 1179, 1279, 1319, 1338, 1343, 767, 772, 1610, 1200, 1276]
Jaccard
       node   jaccard
0    1616.0  0.837209
4    1593.0  0.787234
1    1629.0  0.731707
8    1612.0  0.705882
3    1623.0  0.701754
9    1600.0  0.655738
21   1326.0  0.645161
5    1611.0  0.642857
11   1352.0  0.642857
15   1290.0  0.606061
17   1304.0  0.606061
16   1296.0  0.606061
27   1351.0  0.606061
25   1340.0  0.597015
18   1320.0  0.597015
29   1375.0  0.584906
48   1338.0  0.579710
46   1279.0  0.579710
14   1286.0  0.579710
34   2468.0  0.574468
47   1319.0  0.571429
13   1267.0  0.566038
28   1355.0  0.563636
26   1346.0  0.555556
2    1342.0  0.547945
52   1610.0

In [19]:
conduct(25,4,10)

-------------------
i
0
MSH_Verteilung
[12866.   158.  5548.  8645.]
volunteer
[1616, 1629, 1342, 1623, 1593, 1611, 781, 1606, 1612, 1600, 2470, 1352, 1216, 1267, 1286, 1290, 1296, 1304, 1320, 1322, 1324, 1326, 1329, 1330, 1335, 1340, 1346, 1351, 1355, 1375, 1369, 1282, 1285, 1350, 2468, 10485, 10486, 10487, 10488, 10489, 10490]
Jaccard
       node   jaccard
0    1616.0  0.837209
4    1593.0  0.787234
1    1629.0  0.731707
8    1612.0  0.705882
3    1623.0  0.701754
9    1600.0  0.655738
21   1326.0  0.645161
5    1611.0  0.642857
11   1352.0  0.642857
16   1296.0  0.606061
27   1351.0  0.606061
17   1304.0  0.606061
15   1290.0  0.606061
25   1340.0  0.597015
18   1320.0  0.597015
29   1375.0  0.584906
14   1286.0  0.579710
34   2468.0  0.574468
13   1267.0  0.566038
28   1355.0  0.563636
26   1346.0  0.555556
2    1342.0  0.547945
12   1216.0  0.519481
22   1329.0  0.516667
35  10485.0  0.511111
31   1282.0  0.490566
36  10486.0  0.488889
37  10487.0  0.488889
39  10489.0  0.488889
4

In [20]:
conduct(50,2,10)

-------------------
i
0
MSH_Verteilung
[12866.   158.]
volunteer
[841, 1285, 1616, 1629, 5677, 1267, 1352, 1355, 1361, 1375, 12610, 12611, 1329, 1342, 1593, 1612, 7593, 1200, 1216, 1276, 1279, 1286, 1290, 1296, 1304, 1319, 1320, 1322, 1326, 1338, 1340, 1346, 1351, 1623, 1606, 10486, 10487, 10489, 10490, 1611, 2468, 1607, 1614, 1621, 1625, 1626, 1627, 791, 803, 800, 1589, 1590, 1598, 1600, 1608, 1624, 1628, 1303, 1343, 240, 1324, 1335, 3014, 767, 804, 1273, 1282, 765, 2470, 8469, 781, 924, 4857, 7326, 9061, 1278, 222, 239, 763, 776, 777, 801, 1203, 1330, 1350, 1610, 2887, 3078, 4095, 4096, 7256, 10485, 10488, 1266, 214, 772, 966, 1179, 1369, 2563, 2567, 4898, 5834, 7603, 219, 221, 530, 866, 1146, 1297, 1314, 1844, 1883, 6326, 10372, 10375, 10376, 10377, 10378, 4717, 3556, 11024, 11025, 11669, 11905, 11910, 11911, 11912, 11913, 11914, 11915, 11917, 11918, 11919, 11920, 11921, 3177, 4972, 1832, 1852, 1860, 1869, 1871, 1873, 1881, 1895, 1896, 8704, 8724, 8725, 8727, 8728, 8729, 8730, 8731,

In [21]:
conduct(25,2,10)

-------------------
i
0
MSH_Verteilung
[12866.   158.]
volunteer
[841, 1285, 1616, 1629, 5677, 1267, 1352, 1355, 1361, 1375, 12610, 12611, 1329, 1342, 1593, 1612, 7593, 1200, 1216, 1276, 1279, 1286, 1290, 1296, 1304, 1319, 1320, 1322, 1326, 1338, 1340, 1346, 1351, 1623, 1606, 10486, 10487, 10489, 10490, 1611, 2468, 1607, 1614, 1621, 1625, 1626, 1627, 791, 803, 800, 1589, 1590, 1598, 1600, 1608, 1624, 1628, 1303, 1343, 240, 1324, 1335, 3014, 767, 804, 1273, 1282, 765, 2470, 8469, 781, 924, 4857, 7326, 9061]
Jaccard
       node   jaccard
2    1616.0  0.837209
14   1593.0  0.787234
3    1629.0  0.731707
15   1612.0  0.705882
33   1623.0  0.701754
..      ...       ...
62   3014.0  0.000000
11  12611.0  0.000000
10  12610.0  0.000000
4    5677.0  0.000000
74   9061.0  0.000000

[75 rows x 2 columns]
Jaccard
          node   jaccard
4518    1616.0  0.837209
4515    1593.0  0.787234
4520    1629.0  0.731707
4517    1612.0  0.705882
4519    1623.0  0.701754
...        ...       ...
6536    30

In [22]:
conduct(25,4,100)

-------------------
i
0
MSH_Verteilung
[12866.   158.  5548.  8645.]
volunteer
[1352, 1616, 1629, 1162, 1216, 1285, 1342, 1593, 1267, 1329, 1375, 1612, 791, 803, 1279, 1290, 1296, 1303, 1304, 1320, 1326, 1338, 1340, 1346, 1351, 1623, 1355, 1611, 240, 767, 781, 1276, 1278, 1286, 1343, 1606, 6972, 763, 765, 776, 777, 801, 804, 1273, 1282, 1322, 1330, 1600, 2468, 2470, 3361, 1319, 10446, 1324, 1335, 1369, 214, 1350, 10485, 10486, 10487, 10488, 10489, 10490]
Jaccard
       node   jaccard
1    1616.0  0.837209
7    1593.0  0.787234
2    1629.0  0.731707
11   1612.0  0.705882
25   1623.0  0.701754
..      ...       ...
43   1273.0  0.142857
3    1162.0  0.011628
52  10446.0  0.000000
36   6972.0  0.000000
50   3361.0  0.000000

[64 rows x 2 columns]
Jaccard
          node   jaccard
4518    1616.0  0.837209
4515    1593.0  0.787234
4520    1629.0  0.731707
4517    1612.0  0.705882
4519    1623.0  0.701754
...        ...       ...
6536    3003.0  0.000000
6537    3123.0  0.000000
6538    3134.

In [23]:
conduct(50,2,100)

-------------------
i
0
MSH_Verteilung
[12866.   158.]
volunteer
[670, 767, 777, 840, 841, 1118, 1155, 1285, 1352, 1593, 1616, 1629, 1664, 1832, 1835, 1842, 1848, 1850, 1852, 1854, 1857, 1859, 1860, 1863, 1867, 1869, 1871, 1873, 1880, 1881, 1882, 1888, 1892, 1895, 1896, 1901, 1903, 2322, 2468, 2469, 3835, 3840, 3917, 3932, 4065, 4071, 4072, 4183, 4242, 4338, 4339, 4345, 4349, 4351, 4369, 4374, 5430, 5677, 5708, 5714, 5720, 9546, 11835, 218, 221, 252, 763, 791, 803, 853, 854, 1159, 1170, 1172, 1267, 1269, 1275, 1284, 1305, 1306, 1307, 1317, 1341, 1355, 1361, 1363, 1365, 1375, 1539, 1683, 1805, 1807, 1810, 1811, 1815, 1817, 4559, 6507, 12610, 12611, 15058, 1162, 1196, 1216, 1325, 1329, 1336, 1342, 1372, 1612, 4336, 7593, 8649, 8650, 8651, 8652, 8653, 8654, 8655, 8657, 8658, 8659, 240, 781, 1160, 1200, 1276, 1279, 1286, 1290, 1296, 1304, 1319, 1320, 1322, 1326, 1338, 1340, 1346, 1351, 1611, 1623, 6528, 8138, 9843, 10429, 11950, 765, 772, 804, 1027, 1101, 1282, 1301, 1330, 1343, 1350, 1451

Jaccard
         node   jaccard
10     1616.0  0.837209
9      1593.0  0.787234
11     1629.0  0.731707
109    1612.0  0.705882
141    1623.0  0.701754
...       ...       ...
1111   7326.0  0.000000
1112   7440.0  0.000000
1113   7443.0  0.000000
1114   7459.0  0.000000
2945  16146.0  0.000000

[2946 rows x 2 columns]
Jaccard
          node   jaccard
4518    1616.0  0.837209
4515    1593.0  0.787234
4520    1629.0  0.731707
4517    1612.0  0.705882
4519    1623.0  0.701754
...        ...       ...
6536    3003.0  0.000000
6537    3123.0  0.000000
6538    3134.0  0.000000
6539    9081.0  0.000000
18769  18770.0  0.000000

[18770 rows x 2 columns]
A
[1616.0, 1593.0, 1629.0, 1612.0, 1623.0, 1600.0, 1326.0, 1352.0, 1611.0, 1290.0]
B
[1616.0, 1593.0, 1629.0, 1612.0, 1623.0, 1600.0, 1326.0, 1352.0, 1611.0, 1351.0]
-------------------
i
1
MSH_Verteilung
[ 319. 1218.]
volunteer
[726, 1091, 1410, 2279, 2317, 2626, 2928, 6658, 7231, 7471, 8114, 9134, 9135, 9596, 9937, 10083, 10877, 11377, 15378

Jaccard
         node   jaccard
52     5842.0  0.343137
1453   4121.0  0.320755
51     5838.0  0.317757
33     2592.0  0.305195
24      586.0  0.286885
...       ...       ...
635   15412.0  0.000000
2795   6625.0  0.000000
2796   6626.0  0.000000
2797   7080.0  0.000000
5240  13357.0  0.000000

[5241 rows x 2 columns]
Jaccard
          node   jaccard
3094    5842.0  0.343137
3658    4121.0  0.320755
3092    5838.0  0.317757
2723    2592.0  0.305195
2248     586.0  0.286885
...        ...       ...
7839    7778.0  0.000000
7840    1388.0  0.000000
7841    1390.0  0.000000
7842     821.0  0.000000
18769  18770.0  0.000000

[18770 rows x 2 columns]
A
[5842.0, 4121.0, 5838.0, 2592.0, 586.0, 3367.0, 3875.0, 3358.0, 2599.0, 1039.0]
B
[5842.0, 4121.0, 5838.0, 2592.0, 586.0, 3367.0, 3875.0, 3358.0, 2599.0, 1039.0]
-------------------
i
2
MSH_Verteilung
[14572.   812.]
volunteer
[1933, 1934, 1937, 2066, 2962, 3825, 3829, 3834, 3896, 8133, 8895, 11648, 11675, 12724, 12878, 12879, 12882, 14500, 

In [24]:
conduct(25,2,100)

-------------------
i
0
MSH_Verteilung
[12866.   158.]
volunteer
[670, 767, 777, 840, 841, 1118, 1155, 1285, 1352, 1593, 1616, 1629, 1664, 1832, 1835, 1842, 1848, 1850, 1852, 1854, 1857, 1859, 1860, 1863, 1867, 1869, 1871, 1873, 1880, 1881, 1882, 1888, 1892, 1895, 1896, 1901, 1903, 2322, 2468, 2469, 3835, 3840, 3917, 3932, 4065, 4071, 4072, 4183, 4242, 4338, 4339, 4345, 4349, 4351, 4369, 4374, 5430, 5677, 5708, 5714, 5720, 9546, 11835, 218, 221, 252, 763, 791, 803, 853, 854, 1159, 1170, 1172, 1267, 1269, 1275, 1284, 1305, 1306, 1307, 1317, 1341, 1355, 1361, 1363, 1365, 1375, 1539, 1683, 1805, 1807, 1810, 1811, 1815, 1817, 4559, 6507, 12610, 12611, 15058, 1162, 1196, 1216, 1325, 1329, 1336, 1342, 1372, 1612, 4336, 7593, 8649, 8650, 8651, 8652, 8653, 8654, 8655, 8657, 8658, 8659, 240, 781, 1160, 1200, 1276, 1279, 1286, 1290, 1296, 1304, 1319, 1320, 1322, 1326, 1338, 1340, 1346, 1351, 1611, 1623, 6528, 8138, 9843, 10429, 11950, 765, 772, 804, 1027, 1101, 1282, 1301, 1330, 1343, 1350, 1451

Jaccard
          node   jaccard
4518    1616.0  0.837209
4515    1593.0  0.787234
4520    1629.0  0.731707
4517    1612.0  0.705882
4519    1623.0  0.701754
...        ...       ...
6536    3003.0  0.000000
6537    3123.0  0.000000
6538    3134.0  0.000000
6539    9081.0  0.000000
18769  18770.0  0.000000

[18770 rows x 2 columns]
A
[1616.0, 1593.0, 1629.0, 1612.0, 1623.0, 1600.0, 1326.0, 1611.0, 1352.0, 1304.0]
B
[1616.0, 1593.0, 1629.0, 1612.0, 1623.0, 1600.0, 1326.0, 1352.0, 1611.0, 1351.0]
-------------------
i
1
MSH_Verteilung
[ 319. 1218.]
volunteer
[726, 1091, 1410, 2279, 2317, 2626, 2928, 6658, 7231, 7471, 8114, 9134, 9135, 9596, 9937, 10083, 10877, 11377, 15378, 15967, 157, 171, 351, 394, 586, 588, 1186, 2242, 2244, 2253, 2287, 2320, 2490, 2592, 3144, 3162, 3233, 3293, 3347, 3358, 3455, 3460, 3846, 3884, 4061, 4300, 5080, 5100, 5146, 5674, 5830, 5838, 5842, 5843, 5856, 5875, 5998, 6018, 6040, 6313, 6394, 6622, 7831, 7837, 7922, 8832, 9588, 9852, 11999, 12678, 13139, 17259, 17

Jaccard
         node   jaccard
52     5842.0  0.343137
1453   4121.0  0.320755
51     5838.0  0.317757
33     2592.0  0.305195
24      586.0  0.286885
...       ...       ...
1428   2556.0  0.000000
1426   2133.0  0.000000
1422   2067.0  0.000000
1421   2015.0  0.000000
2921  16524.0  0.000000

[2922 rows x 2 columns]
Jaccard
          node   jaccard
3094    5842.0  0.343137
3658    4121.0  0.320755
3092    5838.0  0.317757
2723    2592.0  0.305195
2248     586.0  0.286885
...        ...       ...
7839    7778.0  0.000000
7840    1388.0  0.000000
7841    1390.0  0.000000
7842     821.0  0.000000
18769  18770.0  0.000000

[18770 rows x 2 columns]
A
[5842.0, 4121.0, 5838.0, 2592.0, 586.0, 3367.0, 3875.0, 3358.0, 2599.0, 1039.0]
B
[5842.0, 4121.0, 5838.0, 2592.0, 586.0, 3367.0, 3875.0, 3358.0, 2599.0, 1039.0]
-------------------
i
2
MSH_Verteilung
[14572.   812.]
volunteer
[1933, 1934, 1937, 2066, 2962, 3825, 3829, 3834, 3896, 8133, 8895, 11648, 11675, 12724, 12878, 12879, 12882, 14500, 